In [1]:
# 内置类型dict的__init__方法和__update__方法会忽略我们覆盖的___setitem__的方法

In [2]:
class DoppeDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [4]:
dd = DoppeDict(one=1) #__init__方法忽略我们覆盖的__setitem__用法
dd

{'one': 1}

In [6]:
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [8]:
dd.update(three = 3) # __update__方法也会忽略自定义的__setitem__方法
dd

{'one': 1, 'two': [2, 2], 'three': 3}

In [13]:
#同样的例子 ： dict子类覆盖的__getitem__方法不会被内置类型的get()调用

In [14]:
class AnswerDict(dict):
    def __getitem__(self, item):
        return 42

In [15]:
ad = AnswerDict(a = 'foo')
ad['a']

42

In [18]:
d = {}
d.update(ad)   #用ad值更新d中的值
d['a']

'foo'

In [19]:
d

{'a': 'foo'}

In [21]:
# 直接子类化内置类型容易出错，因为内置类型的方法通常会忽略用户覆盖的方法
# 用户自定义类应该继承collection模块

In [22]:
import collections

class DoppeDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2)

In [28]:
# 发现调用成功
ad = DoppeDict2(one=1)
ad.update(two=2)
ad

{'one': [1, 1], 'two': [2, 2]}

In [29]:
# 再测试__getitem方法

In [31]:
import collections

class AnswerDict2(collections.UserDict):
    def __getitem__(self, item):
        return 42

In [38]:
# 调用成功
ad = AnswerDict2(one=1)
d = dict()
d.update(ad)

In [39]:
d

{'one': 42}

In [40]:
# 多重继承和方法解析顺序

In [1]:
class A:
    def ping(self):
        print("ping:", self)

class B(A):
    def pong(self):
        print("pong", self)

class C(A):
    def pong(self):
        print("PONG", self)

class D(B, C):

    def ping(self):
        super().ping()
        print('post-ping', self)

    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [9]:
d = D()
d.pong() #这里发现d.pong()调用的是B的

pong <__main__.D object at 0x000002031A9063C8>


In [4]:
C.pong(d) #超类方法可以直接调用

PONG <__main__.D object at 0x000002031A9063C8>


In [5]:
# Python 可以区分d.pong()调用的是哪个方法，是因为python会按特定的顺序遍历继承图

In [6]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [10]:
d.ping()

ping: <__main__.D object at 0x000002031A9063C8>
post-ping <__main__.D object at 0x000002031A9063C8>


In [11]:
# 查看pingpong方法的五个调用
# 第一行   self.ping() 调用的是D的ping方法, 即A的ping（）方法
# 第二行   super().ping(),跳过D类的ping方法，找到A类的ping（）方法
# 第三行   self.pong() 根据__mro__元组的信息，调用的是B的pong方法
# 第四行   super().ping() 根据__mro__元组的信息，调用的是B的pong方法
# 第五行   C.pong()忽略__mro__，找到的是C类实现的pong（）方法
d.pingpong()

ping: <__main__.D object at 0x000002031A9063C8>
post-ping <__main__.D object at 0x000002031A9063C8>
ping: <__main__.D object at 0x000002031A9063C8>
pong <__main__.D object at 0x000002031A9063C8>
pong <__main__.D object at 0x000002031A9063C8>
PONG <__main__.D object at 0x000002031A9063C8>


In [12]:
def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))

In [14]:
import tkinter
print_mro(tkinter.Text)

Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object


In [15]:
# 处理多重继承的

In [16]:
# 1.把接口继承和实现区分开
# 2.使用抽象基类显式表示接口
# 3.通过混入重用代码
# 4.在名称中明确指明混入
# 5.抽象基类可以作为混入，反过来则不成立
# 6.不要子类化多个具体类
# 7.为用户提供聚合类
# 8.“优先使用对象组合， 而不是类继承”